# The Viewer

In [ ]:
import numpy as np
import json
import time
import ipywidgets as widgets
from cad_viewer_widget import AnimationTrack, CadViewer, show, get_sidecar, get_sidecars, get_default, close_sidecar, close_sidecars, open_viewer, set_default

from cad_viewer_widget.utils import numpyify

names = ["hexapod", "hexapod_m", "box", "box1", "boxes", "edges", "faces", "vertices", "pylones", "box-edges", "box-faces", "box-vertices"]
objects = {}
states = {}

for name in names:
    with open(f"../examples/{name}.json", "r") as fd:
        objects[name] = numpyify(json.load(fd))

    with open(f"../examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

# Sidecar handling

## openviewer and add_shapes

In [ ]:
cv2 = open_viewer(
    title="CVW 2",
    anchor="split-right",
    cad_width=640,
    tree_width=250,
    height=480,
    theme="light",
)

In [ ]:
name = "hexapod"
cv2.add_shapes(
    objects[name], 
    states[name],

    #tools=False,
    #ortho=False,
    control="trackball",
    #control="orbit",
    axes=False,
    axes0=False,
    grid=(True,True, False),
    ticks=10,
    transparent=True,
    #black_edges=True,

    normal_len=0,
    default_edge_color="#707070",
    default_opacity=0.5,
    ambient_intensity=0.5,
    direct_intensity=0.3,

    reset_camera=False,
    #position = (865.4844022079983, -276.23389988421786, 335.21716816984906),
    #quaternion = (0.43557639340677845, 0.3648618806188253, 0.4409953598863984, 0.6947460731351442),
    #zoom=0.5,
    timeit=False,
    
    zoom_speed=0.5,
    pan_speed=0.5,
    rotate_speed=1.0,
    js_debug=True,
)

## show

In [ ]:
name = "hexapod"
cv = show(
    objects[name], 
    states[name],
    title="CVW 1",
    height=480,
    cad_width=640,
    tools=True,
)

In [ ]:
name = "boxes"
cv2 = show(
    objects[name], 
    states[name],
    title="CVW 2",
    anchor="split-right",
    height=480,
    cad_width=640,
    
    ortho=False,
    control="orbit",
    axes=True,
    grid=(True,True, False),
    ticks=5,
    transparent=True,
    #black_edges=True,

    normal_len=2,
    default_edge_color="#707070",
    default_opacity=0.5,
    ambient_intensity=0.5,
    direct_intensity=0.3,    
)

In [ ]:
cv.close()
cv.disposed

In [ ]:
cv2.close()
cv2.disposed

# Use default sidecar

In [ ]:
set_default("CVW 1")

In [ ]:
name = "faces"
cv = show(
    objects[name], 
    states[name],
    height=600,
    cad_width=800,
    reset_camera=False
)

In [ ]:
cv.dump_model()

In [ ]:
cv.widget.zoom = 1

In [ ]:
get_sidecar("CVW 1").widget.position

In [ ]:
get_sidecars()

In [ ]:
cv.position, cv.quaternion

In [ ]:
close_sidecars()

# Cell Handling

In [ ]:
name = "hexapod"
cv3 = show(
    objects[name],
    states[name],
    height=600,
    cad_width=800,
    control="trackball",
    tools=True,

    axes=True,
    axes0=True,
    grid=[True, False, True],
    transparent=True,
    black_edges=True,
    ortho=False,
    timeit=True,
#    normal_len=5,
#    reset_camera=False,
)

In [ ]:
cv3.widget.grid

In [ ]:
name = "faces"
cv4 = show(
    objects[name],
    states[name],

    height=600,
    cad_width=800,
)


## Open viewer in cell

In [ ]:
name = "edges"

cv1 = show(
    objects[name],
    states[name], 
    cad_width=640, 
    height=480, 
    ortho=True,
    control="trackball",
    grid=(False, False, False),
    tools=True,
)

In [ ]:
name = "faces"

cv2 = show(
    objects[name],
    states[name], 
    cad_width=640, 
    height=480, 
    ortho=True,
    control="orbit",
    grid=(False, False, False),
    tools=True,
)

In [ ]:
from ipywidgets.embed import embed_minimal_html
# !!! sc.close()
embed_minimal_html('export.html', views=[cv2.widget], title='Widgets export')

# Property access

In [ ]:
cv = open_viewer(
    title = "Examples",
    anchor="right",
    cad_width=640,
    height=480
)
cv.js_debug=True

## Adding shapes

In [ ]:
menu = widgets.Dropdown(
    options=names,
    value=names[0],
    description='Number:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        name = change['new']
        show(    
            objects[name],
            states[name],
            title="Examples",
            control="orbit"
        )

menu.observe(on_change)

show(
    objects[names[0]],
    states[names[0]],
    title="Examples"    
)
menu

# Camera position control

In [ ]:
p0 = cv.position
if cv.control == "trackball":
    q0 = cv.quaternion
else:
    q0 = None
p0, q0

In [ ]:
cv.position = [-580.0, 200.0, 750.0]

if cv.control == "trackball":
    cv.quaternion = [-0.33, 0.026, 0.91, 0.24]

cv.zoom = 0.5

In [ ]:
cv.position = p0

if cv.control == "trackball":
    cv.quaternion = q0
    
cv.zoom = 1

# Widget interaction

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.default_edge_color

In [ ]:
cv.default_edge_color = "#ff0000"

In [ ]:
cv.default_edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.grid = [not g for g in cv.widget.grid]
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges
cv.tools = not cv.tools
cv.ortho = not cv.ortho

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed =1
cv.pan_speed =1
cv.rotate_speed =1

In [ ]:
cv.last_pick

# Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = not cv.clip_intersection

In [ ]:
cv.clip_planes = not cv.clip_planes

In [ ]:
cv.clip_value_0 = 10
cv.clip_value_1 = -50
cv.clip_value_2 = 40

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.select_tree()

# Rotations

In [ ]:
for i in range(10):
    cv.rotate_x(1)
    cv.rotate_y(3)
    cv.rotate_z(5)
    time.sleep(0.1)

In [ ]:
for i in range(10):
    cv.rotate_z(-5)
    cv.rotate_y(-3)
    cv.rotate_x(-1)
    time.sleep(0.1)    

In [ ]:
cv.rotate_up(30)
cv.rotate_left(30)

In [ ]:
cv.rotate_left(-30)
cv.rotate_up(-30)

# Animation

In [ ]:
name = "hexapod"
cv = show(
    objects[name],
    states[name],
    title="Animation",
    height=600,
    cad_width=800,
    control="trackball",
    tools=True,

    axes=True,
    axes0=True,
    grid=[True, False, True],
)

In [ ]:
import numpy as np
horizontal_angle = 25

leg_names = {
    "right_back", "right_middle", "right_front", 
    "left_back",  "left_middle", "left_front",
    
}

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1).tolist()
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

leg_group = ("left_front", "right_middle", "left_back")

In [ ]:
tracks = []

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(f"/bottom/{name}", "rz", *horizontal(4, "middle" in name)))

cv.animate(5)
cv.play()

In [ ]:
cv.stop()

In [ ]:
for name in leg_names:
    # move lower leg
    cv.add_track(AnimationTrack(f"/bottom/{name}/lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)))
    
cv.animate(2)
cv.play()

In [ ]:
cv.clear_tracks()

# Embedding (experimental)

In [ ]:
from cad_viewer_widget.embed import embed

name = "box1"
sh = json.dumps(objects[name], default=serializer)
st = json.dumps(states[name])

embed(sh, st, tools=False, axes=True, transparent=True)